In [5]:
import pandas as pd


In [7]:
!pip install wikipedia
!pip install requests
!pip install BeautilfulSoup
!pip install time


  Could not find a version that satisfies the requirement BeautilfulSoup (from versions: )
No matching distribution found for BeautilfulSoup


  Could not find a version that satisfies the requirement time (from versions: )
No matching distribution found for time


In [1]:
!pip install lxml

In [41]:
import requests
from bs4 import BeautifulSoup
import bs4
import time
import numpy as np
import re
#the goal is scrapping wikipedia in order to obtain information about citrus fruit
#We take the description section of each one and then we will extract main info.


In [79]:
page_citrus = 'https://en.wikipedia.org/wiki/List_of_citrus_fruits'
page_salads = 'https://en.wikipedia.org/wiki/List_of_leaf_vegetables'
page_spices = 'https://en.wikipedia.org/wiki/List_of_culinary_herbs_and_spices'

#A function that gets the URL of the page to be scraped
#,gets the html content and uses BeautifulSoup to parse html content

def make_soup(link):
    get_page = requests.get(link)
    html = get_page.content
    soup = BeautifulSoup(html, 'html.parser')
    return  soup
#print(make_soup(page_spices))
#this function create a list with all the link of the foods in a wikipedia list.
def make_link_list(wiki_page_to_scrap):
    link_table = []
    soup = make_soup(wiki_page_to_scrap)
    table = soup.find('table',{'class':'wikitable sortable'})
    print(type(table))
    if isinstance(table , bs4.element.Tag):        
        species_table = table.find_all('a')      
        for link in species_table:
            species = link.get('title')
            species = str(species)
            if re.search('None', species) == None and re.search('Wikipedia' , species) == None :  
                link_table.append(species)
    else:
        table = soup.find('div' , {'class':'div-col'})
        species_table = table.find_all('li')
        for text in species_table:
            print(type(text))
            species = text.find_all('#text')
            
            link_table.append(species)
        
        
        print(link_table)
  
    return link_table
    
    
#list_citrus = make_link_list(page_citrus)
#print(list_citrus[:10])

#list_salad = make_link_list(wiki_page_salads) 
#print(list_salad[:10])
### Attention latin name were scrapped => tackle the traduction between latin & common name
## maybe change in the "make_link_list"

list_spices = make_link_list(page_spices)
#print(list_spices)


<class 'NoneType'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
<class 'bs4.element.Tag'>
[[], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [41]:
# Let's create a dictionary with the different categories of vegetables. 
#Categories will be: 
    #Stem Vegetables
    #Leaves Vegetables
    #Flowers Vegetables
    #Bulb Vegetables
    #Beans
    #Roots Vegetables
    #Tubers Vegetables
    #Fruits Vegetables
    #Fungi Vegetables


vegetable_dict = {'Stem' : 0 , 'Leaves' : 0 , 'Flowers' : 0 , 'Bulb' : 0 , 'Beans' : 0 , 'Roots' : 0 , 'Tubers' : 0 , 'Fruits' : 0 , 'Fungi': 0 }


In [29]:
print(type(5))


<class 'int'>
